In [4]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate 
#bitsandbytes

In [ ]:
!pip install C:/triton-2.1.0-cp310-cp310-win_amd64.whl
!pip install C:/bitsandbytes-0.43.0.dev0-cp310-cp310-win_amd64.whl
!pip install C:/deepspeed-0.13.1+unknown-py3-none-any.whl

In [2]:
!pip freeze

accelerate==0.30.1
aiohttp==3.9.5
aiosignal==1.3.1
asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1698341106958/work
async-timeout==4.0.3
attrs==23.2.0
backcall @ file:///home/conda/feedstock_root/build_artifacts/backcall_1592338393461/work
bitsandbytes==0.43.1
certifi==2024.2.2
charset-normalizer==3.3.2
colorama @ file:///home/conda/feedstock_root/build_artifacts/colorama_1666700638685/work
comm @ file:///home/conda/feedstock_root/build_artifacts/comm_1710320294760/work
datasets==2.19.1
debugpy @ file:///D:/bld/debugpy_1707444604970/work
decorator @ file:///home/conda/feedstock_root/build_artifacts/decorator_1641555617451/work
dill==0.3.8
einops==0.8.0
executing @ file:///home/conda/feedstock_root/build_artifacts/executing_1698579936712/work
faiss==1.7.4
filelock==3.14.0
frozenlist==1.4.1
fsspec==2024.3.1
huggingface-hub==0.23.1
idna==3.7
ijson==3.2.3
importlib_metadata @ file:///home/conda/feedstock_root/build_artifacts/importlib-metadata_1710971335535/work
i

In [1]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import tqdm as notebook_tqdm

model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = checkpoint_dir + "/litm_model", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 4096,
    dtype = None,
    model_name = "Paoloc99/litm_model", # YOUR MODEL YOU USED FOR TRAINING
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

d:\anaconda3\envs\unsloth_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA GeForce RTX 3060 Ti. Max memory: 7.999 GB. Platform = Windows.
O^O/ \_/ \    Pytorch: 2.3.0. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth: unsloth/tinyllama-bnb-4bit can only handle sequence lengths of at most 2048.
But with kaiokendev's RoPE scaling of 2.0, it can be magically be extended to 4096!
d:\anaconda3\envs\unsloth_env\lib\site-packages\unsloth\models\llama.py:1022: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\cb\pytorch_1000000000000\work\c10/cuda/CUDAAllocatorConfig.h:28.)
  self.register_buffer("cos_cached", emb.cos().to(dtype=dtype, device=device, non_blocking=True), persistent=False)
Unsloth 2024.8 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.


In [2]:
from datasets import load_dataset
eval_dataset = load_dataset("Paoloc99/dataset", split="test[:100]")


In [3]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import re

# Assumendo che test_dataset sia una lista di dizionari con 'prompt' e 'true_text'
batch_size = 16  # Numero di esempi da processare in parallelo

def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

def generate_batch_texts(prompts, model, tokenizer, max_length=4096):
    
    # Tokenizzazione dei prompt
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to('cuda')
    
    # Generazione del testo
    outputs = model.generate(**inputs, max_length=max_length)
    
    # Decodifica del testo generato
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    # Rimozione del testo prima di "Answer: " in ogni prompt
    modified_generated_output = []
    for text in generated_texts:
        modified_output = text.split("Answer: ", 1)[1]
        modified_output = remove_punctuation(modified_output.lower()).strip()
        modified_generated_output.append(modified_output)

    answers_ids = tokenizer(modified_generated_output, return_tensors="pt", padding=True, truncation=True).to('cuda')
    answers_ids = answers_ids['input_ids']

    # Rimozione degli elementi 0 e 1 da ciascun tensore nella lista
    cleaned_input_ids = []
    for tensor in answers_ids:
        # Filtra il tensore per mantenere solo gli ID diversi da 0 e 1
        filtered_tensor = tensor[tensor > 1]
        cleaned_input_ids.append(filtered_tensor)

    return cleaned_input_ids, modified_generated_output

def generate_true_ids(prompts, model, tokenizer, max_length=4096):
    # Tokenizzazione dei prompt
    prompts = [remove_punctuation(prompt.lower()) for prompt in prompts]
    outputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to('cuda')
    outputs = outputs['input_ids']
    
    # Rimozione degli elementi 0 e 1 da ciascun tensore nella lista
    cleaned_input_ids = []
    for tensor in outputs:
        # Filtra il tensore per mantenere solo gli ID diversi da 0 e 1
        filtered_tensor = tensor[tensor > 1]
        cleaned_input_ids.append(filtered_tensor)
    
    return cleaned_input_ids

# Dataloader per creare batch di input
data_loader = DataLoader(eval_dataset, batch_size=batch_size)

# List per salvare i risultati
prompts = []
generated_texts = []
# generated_ids = []
true_texts = []
# true_ids = []

# Loop attraverso i batch
for batch in tqdm(data_loader):
    prompts.extend(batch['prompt'])  # Lista di prompt nel batch
    true_texts.extend([remove_punctuation(el.lower()) for el in batch['completion']])  # Salva i true_texts per questo batch
    
    # Genera il testo per l'intero batch
    batch_generated_ids, batch_generated_texts = generate_batch_texts(batch['prompt'], model, tokenizer)
    batch_true_ids = generate_true_ids(batch['completion'], model, tokenizer)

    # Salva i testi generati
    generated_texts.extend(batch_generated_texts)
    # generated_ids.extend(batch_generated_ids)
    # true_ids.extend(batch_true_ids)

  0%|          | 0/7 [00:00<?, ?it/s]d:\anaconda3\envs\unsloth_env\lib\site-packages\unsloth\models\llama.py:407: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  A = scaled_dot_product_attention(Q, K, V, attn_mask = attention_mask, is_causal = False)
100%|██████████| 7/7 [01:40<00:00, 14.38s/it]


In [10]:
index = -1
print(generated_texts[index])
print(true_texts[index])
# print(generated_ids[index])
# print(true_ids[index])

december 1  1996
december 1


In [8]:
from collections import Counter
import evaluate
from bert_score import score

def match_tokens(generated_texts_list, true_texts_list):
    tp = 0  # Veri positivi
    fp = 0  # Falsi positivi
    fn = 0  # Falsi negativi

    # Itera su ogni coppia di tensori nella lista
    for true_list, generated_list in zip(true_texts_list, generated_texts_list):        
        # Conta la frequenza di ciascun token nelle liste
        true_counter = Counter(true_list)
        generated_counter = Counter(generated_list)
        
        # Calcola i veri positivi e i falsi negativi
        for token in true_counter:
            if token in generated_counter:
                tp += min(true_counter[token], generated_counter[token])
                fn += max(0, true_counter[token] - generated_counter[token])
            else:
                fn += true_counter[token]
        
        # Calcola i falsi positivi
        for token in generated_counter:
            if token not in true_counter:
                fp += generated_counter[token]
            elif generated_counter[token] > true_counter[token]:
                fp += generated_counter[token] - true_counter[token]
    
    return tp, fp, fn

def compute_partial_match(generated_texts, true_texts):
    pm = 0

    # Itera su ogni coppia di tensori nella lista
    for true_list, generated_list in zip(generated_texts, true_texts):        
        # Conta la frequenza di ciascun token nelle liste
        true_counter = Counter(true_list)
        generated_counter = Counter(generated_list)

        # Calcola i falsi positivi
        for token in generated_counter:
            if token in true_counter:
                pm += 1
                break
    
    return pm/len(true_texts)

def compute_partial_match_jaccard(generated_texts, true_texts):
    pmj = 0

    # Itera su ogni coppia di tensori nella lista
    for true_list, generated_list in zip(true_texts, generated_texts):     
        true_set = set(true_list)   
        generated_set = set(generated_list)
        
        pmj += len(true_set.intersection(generated_set)) / len(true_set.union(generated_set))
        
    return pmj/len(true_texts)

def compute_exact_match(generated_texts, true_texts):
    em = sum([1 if p == l else 0 for p, l in zip(generated_texts, true_texts)]) / len(true_texts)
    return em

def compute_rouge(generated_texts, true_texts):
    rouge = evaluate.load('rouge')
    results = rouge.compute(predictions=generated_texts,
                            references=true_texts,
                            use_aggregator=True)
    return results

def compute_bleu(generated_texts, true_texts):
    bleu = evaluate.load("bleu")
    true_list = [[text] for text in true_texts]
    results = bleu.compute(predictions=generated_texts, references=true_list)
    return results['bleu']

def compute_bert_score(generated_texts, true_texts):
    P, R, F1 = score(generated_texts, true_texts, lang='en', verbose=True)
    return P.mean().item(), R.mean().item(), F1.mean().item()

def compute_metrics(generated_texts, true_texts):
    # Calculate Exact Match (EM)
    em = compute_exact_match(generated_texts, true_texts) 

    # Calculate Partial Match (PM)
    pm = compute_partial_match(generated_texts, true_texts)

    # Calculate Partial Match Jaccard (PMJ)
    pmj = compute_partial_match_jaccard(generated_texts, true_texts)

    # Calculate F1
    tp, fp, fn = match_tokens(generated_texts, true_texts)

    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    # Calculate ROUGE
    rouge_dict = compute_rouge(generated_texts, true_texts)

    # Calculate BLEU
    bleu = compute_bleu(generated_texts, true_texts)

    # Compute Precision, Recall and F1 via BERT Score
    bert_precision, bert_recall, bert_f1 = compute_bert_score(generated_texts, true_texts)

    return {'exact_match' : em, 'partial_match' : pm, 'jaccard_partial_match' : pmj, 
            'precision' : precision, 'recall' : recall, 'f1' : f1,
            'rouge1' : rouge_dict['rouge1'], 'rouge2' : rouge_dict['rouge2'], 'rougeL' :rouge_dict['rougeL'],
            'rougeLsum': rouge_dict['rougeLsum'], 'bleu' : bleu,
            'bert_precision' : bert_precision, 'bert_recall' : bert_recall, 'bert_f1' : bert_f1}

# metrics = compute_metrics(generated_ids, true_ids)
metrics = compute_metrics(generated_texts, true_texts)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 3/3 [00:11<00:00,  3.68s/it]


computing greedy matching.


100%|██████████| 2/2 [00:00<00:00, 32.00it/s]

done in 11.11 seconds, 9.00 sentences/sec


In [9]:
metrics

{'exact_match': 0.51,
 'partial_match': 0.98,
 'jaccard_partial_match': 0.7867234848484849,
 'precision': 0.7995512341062079,
 'recall': 0.7842993396918562,
 'f1': 0.7918518518518517,
 'rouge1': 0.6902316017316016,
 'rouge2': 0.45607142857142857,
 'rougeL': 0.6883867243867242,
 'rougeLsum': 0.6913152958152957,
 'bleu': 0.43485823399845347,
 'bert_precision': 0.9416860938072205,
 'bert_recall': 0.9391191005706787,
 'bert_f1': 0.9399850964546204}

In [2]:
!wandb login 372f5c298afc4be9b40dd7b97523d394c3d30d05

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\paolo\_netrc


In [5]:
import wandb
# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="lost-in-the-middle"
# set the wandb project where this run will be logged
os.environ["WANDB_NOTEBOOK_NAME "]="lost-in-the-middle"
# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="true"
# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"
wandb.login()

True

In [23]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3060 Ti. Max memory = 7.999 GB.
7.273 GB of memory reserved.


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
index = 3
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    eval_dataset[index]["prompt"]
], return_tensors = "pt").to("cuda")

# print(eval_dataset[2]["prompt"])
outputs = model.generate(**inputs, use_cache = True)
print(tokenizer.batch_decode(outputs)[0])
print("-------------")
print("Risposta esatta: ", eval_dataset[index]["completion"])